# FAST-EM import
---
**Author**: Ryan Lane  
**Date**: 6 April 2022

#### Overview
Quickly imports a FAST-EM project to `render-ws`.

Assumes data is stored as

Stack | Filepath
- | -
raw | `/.../asm_service/{date}/{project}/{section}/{row}_{col}_{zoom}.tiff`
corrected | `/.../asm_service/{date}/{stack}/{section}/corrected/{row}_{col}_{zoom}.tiff`

and outputs mipmaps of each field's pyramidal tiff to

Stack | Filepath
- | -
raw | `/.../{project}/raw/{section}/{row}_{col}/{zoom}.tif`
corrected | `/.../{project}/corrected/{section}/{row}_{col}/{zoom}.tif`

**Warning**:
Check filepath tree carefully before executing.

In [5]:
from pathlib import Path
import re
from ruamel.yaml import YAML

from tqdm.notebook import tqdm
from bs4 import BeautifulSoup as Soup
import numpy as np
import pandas as pd
from tifffile import TiffFile

import renderapi
import icatapi
from renderapi.transform import AffineModel

ModuleNotFoundError: No module named 'icatapi'

In [ ]:
# Indirectly enable autocomplete
%config Completer.use_jedi = False

# pandas display options
pd.set_option('display.max_colwidth', 20)

#### Connect to `render-ws`

In [21]:
# render parameters
owner = 'akievits'
date = '2022-06-10'
project = '20220610_UMCU'

# Create a renderapi.connect.Render object
render_connect_params = {
    'host': 'sonic.tnw.tudelft.nl',
    'port': 8080,
    'owner': owner,
    'project': project,
    'client_scripts': '/home/catmaid/render/render-ws-java-client/src/main/scripts',
    'memGB': '2G',
}
render = renderapi.connect(**render_connect_params)
render.make_kwargs()

{'host': 'http://sonic.tnw.tudelft.nl',
 'port': 8080,
 'owner': 'akievits',
 'project': '20220610_UMCU',
 'client_scripts': '/home/catmaid/render/render-ws-java-client/src/main/scripts',
 'client_script': '/home/catmaid/render/render-ws-java-client/src/main/scripts/run_ws_client.sh',
 'memGB': '2G'}

#### Import and export directories

In [22]:
# Import from
dir_FASTEM = Path(f'/long_term_storage/asm_storage/asm_service/{date}/{project}')
# Export to
dir_project = Path(f"/long_term_storage/{owner}/FAST-EM/{project}/")

# Stack directory
!ls -l $dir_FASTEM

total 16
drwxrwxrwx 3 asmftp asmftp 4096 Jun 10 11:31 EPON_80nm_s001
drwxrwxrwx 3 asmftp asmftp 4096 Jun 10 14:51 EPON_80nm_s002


#### Section mapping

In [23]:
# Assume subdirectories of FAST-EM directories are different sections (durr)
dir_sections = sorted([dir_ for dir_ in dir_FASTEM.iterdir() if dir_.is_dir()])
d_sections = {i: d.name for i, d in enumerate(dir_sections)}
d_sections

# Drop sections that aren't needed
# d_sections.pop(0)
# d_sections.pop(1)

{0: 'EPON_80nm_s001', 1: 'EPON_80nm_s002'}

## 1) Create mipmaps
---

#### Filepath layout
Mimaps of each FAST-EM field are output to
`{dir_project}/{stack}/{sectionId}/{row}_{col}/{zoom}.tif` e.g.

In [25]:
import fastemworkflow

ModuleNotFoundError: No module named 'fastemworkflow'

In [10]:
from fastemworkflow.importdata import create_mipmaps

ModuleNotFoundError: No module named 'fastemworkflow'

#### Raw

In [91]:
# Loop through section directories
for z, dir_section in tqdm(enumerate(dir_sections),
                           total=len(dir_sections)):

    # Loop through tiffs in each section
    for fp in tqdm(list(dir_section.glob('[0-9]*_[0-9]*_0.tiff'))):

        # Read tiff
        tiff = TiffFile(fp)
        # Extract metadata
        metadata = {tag.name: tag.value for tag in tiff.pages[0].tags}
        # Infer row, col
        row, col = [int(i) for i in re.findall(r'\d+', fp.stem)][:2]

        # Set directory to output mipmaps
        dir_mipmaps = dir_project / 'raw' / dir_section.name / f"{row:03d}_{col:03d}"
        dir_mipmaps.mkdir(parents=True, exist_ok=True)
        # Create mipmaps
        create_mipmaps(tiff, dir_mipmaps, metadata)    

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/121 [00:00<?, ?it/s]

  0%|          | 0/121 [00:00<?, ?it/s]

#### Corrected

In [ ]:
# Loop through section directories
for z, dir_section in tqdm(enumerate(dir_sections),
                           total=len(dir_sections)):

    # Loop through tiffs in each section
    for fp in tqdm(list(dir_section.glob('corrected/[0-9]*_[0-9]*_0.tiff'))):

        # Read tiff
        tiff = TiffFile(fp)
        # Extract metadata
        metadata = {tag.name: tag.value for tag in tiff.pages[0].tags}
        # Infer row, col
        row, col = [int(i) for i in re.findall(r'\d+', fp.stem)[:2]]

        # Set directory to output mipmaps
        dir_mipmaps = dir_project / 'corrected' / dir_section.name / f"{row:03d}_{col:03d}"
        dir_mipmaps.mkdir(parents=True, exist_ok=True)
        # Create mipmaps
        create_mipmaps(tiff, dir_mipmaps, metadata)    

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/121 [00:00<?, ?it/s]

## 2) Create tile specifications
---

#### Filepath layout
Reminder that mimaps of each FAST-EM field are output to
`{dir_project}/{stack}/{sectionId}/{row}_{col}/{zoom}.tif`

In [ ]:
import json
from renderapi.image_pyramid import ImagePyramid, MipMapLevel

In [ ]:
# Parameters
overlap = 10  # % -- guess

# Collect tile specifications
tile_dicts = []
# Loop through stack directories
for dir_stack in tqdm(list(dir_project.iterdir())):
    # Set stack name ('raw' or 'corrected')
    stack = dir_stack.name

    # Loop through section directories
    for z, sectionId in tqdm(d_sections.items()):

        # Loop through mipmap directories within each section
        dir_section = dir_stack / sectionId
        for dir_mipmap in tqdm(list(dir_section.glob('[0-9]*_[0-9]*')),
                               leave=False):

            # Read base-level tiff
            fp = dir_mipmap / '0.tif'
            tiff = TiffFile(fp)
            # Parse tiff tags for metadata
            md = json.loads(tiff.pages[0].description)
            # Infer row, col
            row, col = [int(i) for i in re.findall(r'\d+', fp.parent.name)[:2]]
            # Set translation based on overlap guess
            x0 = col * (1 - overlap/100) * md['ImageWidth']
            y0 = row * (1 - overlap/100) * md['ImageLength']

            # Create nested MipMapLevels
            mmls = []
            for mmfp in sorted(dir_mipmap.glob('[0-9].tif')):
                level = mmfp.stem
                imageUrl = f"https://sonic.tnw.tudelft.nl{mmfp.as_posix()}"
                mml = MipMapLevel(level, imageUrl=imageUrl)
                mmls.append(mml)
            # Create ImagePyramid from MipMapLevels
            ip = ImagePyramid({m.level: m.mipmap for m in mmls})

            # Handle missing DateTime metadata in corrected tiffs
            try:
                acqtime = pd.to_datetime(md['DateTime'])
            except KeyError:
                acqtime = -1

            # Build up tile specification
            ts = {}
            ts['stack'] = stack
            ts['sectionId'] = sectionId
            ts['z'] = z
            ts['tileId'] = f'{stack[:3]}-S{z:03d}-{row:03d}x{col:03d}'
            ts['acqtime'] = acqtime
            ts['width'] = md['ImageWidth']
            ts['height'] = md['ImageLength']
            ts['imageRow'] = row
            ts['imageCol'] = col
            ts['imagePyramid'] = ip
            ts['minint'] = 0
            ts['maxint'] = 65535
            ts['tforms'] = [AffineModel(B0=x0, B1=y0)]
            tile_dicts.append(ts)

#### Create stack DataFrames

In [ ]:
# Create DataFrame from list of tile specifications
df_stacks = pd.DataFrame(tile_dicts)

# Sneak peak
df_stacks.groupby('stack')\
         .apply(lambda x: x.sample(3))

### Set intensity levels
Sample `n` images/section to determine reasonable min/max intensity values.

In [ ]:
# Set parameters
n = 10          # sample size (per section)
pcts = (1, 99)  # % for intensity clipping
stacks = df_stacks['stack'].unique().tolist()
z_values = df_stacks['z'].unique().tolist()

# Loop through stacks
for stack, df_stack in tqdm(df_stacks.groupby('stack'),
                            total=len(stacks)):

    # Loop through sections
    for z, tileset in tqdm(df_stack.groupby('z'),
                           total=len(z_values),
                           leave=False):

        # Sample filepaths
        fps = tileset.sample(n)['imagePyramid']\
                      .apply(lambda x: x[0]['imageUrl'])\
                      .tolist()

        # Collect min/max intensity values
        minints = []
        maxints = []
        # Loop through sample tiles
        for fp in tqdm(fps, leave=False):

            # Load tiff image
            fp_tiff = fp.split('.nl')[1]
            tiff = TiffFile(fp_tiff)
            image = tiff.asarray()

            # Get intensity percentiles
            minint, maxint = np.percentile(image, pcts)
            minints.append(minint)
            maxints.append(maxint)

        # Set min/max intensity
        df_stacks.loc[(df_stacks['stack'] == stack) &\
                      (df_stacks['z'] == z), 'minint'] = np.mean(minints, dtype=int)
        df_stacks.loc[(df_stacks['stack'] == stack) &\
                      (df_stacks['z'] == z), 'maxint'] = np.mean(maxints, dtype=int)

# Sneak peak
df_stacks.groupby('stack')\
         .apply(lambda x: x.sample(4))

## 3) Upload stack to `render-ws`
---

In [ ]:
from renderapi.render import get_stacks_by_owner_project
from renderapi.tilespec import TileSpec, get_tile_specs_from_stack
from renderapi.stack import create_stack, set_stack_state
from renderapi.client import import_tilespecs

In [ ]:
def upload_stack_DataFrame(df, render, name=None,
                           stackResolutionX=None,
                           stackResolutionY=None,
                           stackResolutionZ=None):
    """Creates a `render-ws` stack from given DataFrame

    Parameters
    ----------
    df : `pd.DataFrame`
        DataFrame of tile data
    render : `renderapi.render.RenderClient`
        `render-ws` instance
    name : str
        Name of stack
        Looks for 'stack' column in `df` if not provided
    """
    # Set stack name
    if name is None:
        stack = df.iloc[0]['stack']
    else:
        stack = name

    # Loop through tiles
    out = f"Creating tile specifications for \033[1m{stack}\033[0m..."
    print(out)
    tile_specs = []
    for i, tile in df.iterrows():
        # Create `TileSpec`s
        ts = TileSpec(**tile.to_dict())
        # Ensure integer min, max intensity
        ts.minint = int(tile['minint'])
        ts.maxint = int(tile['maxint'])
        # Collect `TileSpec`s
        tile_specs.append(ts)

    # Create stack
    create_stack(stack=stack,
                 stackResolutionX=stackResolutionX,
                 stackResolutionY=stackResolutionY,
                 stackResolutionZ=stackResolutionZ,
                 render=render)

    # Import TileSpecs to render
    out = f"Importing tile specifications to \033[1m{stack}\033[0m..."
    print(out)
    import_tilespecs(stack=stack,
                     tilespecs=tile_specs,
                     render=render)

    # Close stack
    set_stack_state(stack=stack,
                    state='COMPLETE',
                    render=render)
    out = f"Stack \033[1m{stack}\033[0m created successfully."
    print(out)

In [ ]:
# Loop through stacks
for stack, df_stack in tqdm(df_stacks.groupby('stack')):

    # Set stack resolution
    Rx = 4
    Ry = 4
    Rz = 100

    # Create stacks
    upload_stack_DataFrame(df=df_stack,
                           name=stack,
                           stackResolutionX=Rx,
                           stackResolutionY=Ry,
                           stackResolutionZ=Rz,
                           render=render)

#### Inspect

In [ ]:
icatapi.plot_tile_map(stacks,
                      render=render)

## 4) Montage
---

Parse transformation data from `.../corrected/montage.xml` file.

In [ ]:
# Initialize stacks DataFrame
df_stacks = pd.DataFrame()

# Loop through stacks
for stack in tqdm(stacks):

    # Create stack DataFrame
    df_stack = icatapi.create_stack_DataFrame(stack=stack,
                                              render=render)
    # Edit stack name
    df_stack['stack'] = stack + '_montaged'

    # Loop through section directories
    for z, dir_section in enumerate(dir_sections):

        # Parse montage.xml file
        fp = dir_section / 'corrected/montage.xml'
        soup = Soup(fp.read_text(), 'lxml')

        # Loop through "patches"
        for patch in soup.find_all('t2_patch'):

            # Parse transform data
            M00, M10, M01, M11, B0, B1 = [float(i) for i in re.findall(
                r'-?[\d.]+(?:[Ee]-?\d+)?', patch['transform'])]
            A = AffineModel(M00, M01, M10, M11, B0, B1)

            # Infer stack index
            row, col = [int(i) for i in patch['title'].split('_')[:2]]
            i = df_stack.loc[(df_stack['z'] == z) &\
                             (df_stack['imageRow'] == row) &\
                             (df_stack['imageCol'] == col)].index.item()
            df_stack.at[i, 'tforms'] = [A]

    # 
    df_stacks = pd.concat([df_stacks, df_stack])

# Sneak peak
df_stacks.groupby('stack')\
         .apply(lambda x: x.sample(4))

#### Upload montage stacks

In [ ]:
# Loop through stacks
for stack, df_stack in tqdm(df_stacks.groupby('stack')):

    # Set stack resolution
    Rx = 4
    Ry = 4
    Rz = 100

    # Create stacks
    upload_stack_DataFrame(df=df_stack,
                           name=stack,
                           stackResolutionX=Rx,
                           stackResolutionY=Ry,
                           stackResolutionZ=Rz,
                           render=render)

#### Inspect

In [ ]:
stacks_2_plot = renderapi.render.get_stacks_by_owner_project(render=render)
icatapi.plot_tile_map(stacks_2_plot,
                      render=render)

In [ ]:
icatapi.plot_stacks(stacks_2_plot,
                    maxTileSpecsToRender=1000,
                    render=render)